In [7]:

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [18]:
def find_single_occurrences(x, y):
    occurrences = {}

    # Count the occurrences of each element in y
    for elem in y:
        if elem in occurrences:
            occurrences[elem] += 1
        else:
            occurrences[elem] = 1

    # Find elements with only one occurrence
    single_occurrences = [elem for elem in y if occurrences[elem] == 1]

    # Delete elements and their corresponding elements from x and y
    for elem in single_occurrences:
        index = y.index(elem)
        del x[index]
        del y[index]

    return x, y

In [8]:
# Fastai
from fastai.vision.all import *
from fastbook import *
from fastai.vision.data import ImageDataLoaders

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Pandas
import pandas as pd

# Pytorch
import torchvision

# Sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [11]:
from sklearn.model_selection import train_test_split
# Load the .npz file

data_imgs = np.load('kkanj-imgs.npz')
data_labels = np.load('kkanji-labels.npz')

# Access the arrays stored in the .npz files
imgs = data_imgs['arr_0']
labels = data_labels['arr_0']

# Splitting the data into training and testing sets
#Consider reducing data set to stratify

x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.1, random_state=42)

# Printing the sizes of the resulting sets
print("Size of x_train:", len(x_train))
print("Size of y_train:", len(y_train))
print("Size of x_test:", len(x_test))
print("Size of y_test:", len(y_test))

Size of x_train: 126381
Size of y_train: 126381
Size of x_test: 14043
Size of y_test: 14043


In [12]:
# Training Parameters
VERSION = 'kkanjiHalf150_v7.3'
IMAGESIZE = 224
VALIDATION_SIZE = 0.3
RANDOM_STATE = 2018

model_name = 'resnet152'
model = getattr(torchvision.models, model_name)

# path definieren
path = Path('/data/')

In [13]:
# Load Function
def load(f):
    return np.load(f, allow_pickle=True)['arr_0']

In [15]:

# StratifiedShuffleSplit for the dataloaders object. This ensures that test and validation dataset include all Kanji characters
find_single_occurences()


from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=VALIDATION_SIZE, random_state=RANDOM_STATE)

for train_index, test_index in sss.split(x_train, y_train):
    X_trainfin, X_testfin = x_train[train_index], x_train[test_index]
    y_trainfin, y_testfin = y_train[train_index], y_train[test_index]

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [19]:
# Code is based on http://nghiaho.com/?p=2741
def make_dataloaders_from_numpy_data(image, label):
    def pass_index(idx):
        return idx

    def get_x(i):
        return image[i]

    def get_y(i):
        return label[i]

    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=pass_index,
        splitter=IndexSplitter(test_index), # stratified split
        get_x=get_x,
        get_y=get_y,
        item_tfms = Resize(IMAGESIZE),
        batch_tfms=aug_transforms(do_flip=False, p_affine=1)
        )

    # pass in a list of index
    num_images = image.shape[0]
    dls = dblock.dataloaders(list(range(num_images)))

    return dls

dls = make_dataloaders_from_numpy_data(x_train, y_train)

NameError: name 'test_index' is not defined

In [ ]:
# Create a Learner object
learn = cnn_learner(dls, model, metrics=error_rate)

In [ ]:

# Train a model for 10 epochs
learn.fit_one_cycle(10, lr_max=0.01)
learn.save(str(path) + '/model/' + VERSION)